In [ ]:
import pandas as pd

# Read CSV Dataset
df_vgs = pd.read_csv('../data/video_games_sales_with_ratings.csv')

# Data sample
df_vgs.head()


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [273]:
# Make all column names lowercase
df_vgs.columns = df_vgs.columns.str.lower()


In [274]:
# Define the mapping for the descriptive display name (Title Case)
platform_display_map = {
    'Wii': 'Wii',
    'NES': 'Nintendo Entertainment System',
    'GB': 'Game Boy',
    'DS': 'Nintendo DS',
    'X360': 'Xbox 360',
    'PS3': 'PlayStation 3',
    'PS2': 'PlayStation 2',
    'SNES': 'Super Nintendo Entertainment System',
    'GBA': 'Game Boy Advance',
    'PS4': 'PlayStation 4',
    '3DS': 'Nintendo 3DS',
    'N64': 'Nintendo 64',
    'PS': 'PlayStation',
    'XB': 'Xbox',
    'PC': 'PC',
    '2600': 'Atari 2600',
    'PSP': 'PlayStation Portable',
    'XOne': 'Xbox One',
    'WiiU': 'Wii U',
    'GC': 'GameCube',
    'GEN': 'Sega Genesis',
    'DC': 'Dreamcast',
    'PSV': 'PlayStation Vita',
    'SAT': 'Sega Saturn',
    'SCD': 'Sega CD',
    'WS': 'WonderSwan',
    'NG': 'Neo Geo',
    'TG16': 'TurboGrafx-16',
    '3DO': '3DO',
    'GG': 'Sega Game Gear',
    'PCFX': 'PC-FX'
}


In [275]:
# Map the original 'platform' column to the new display names
df_vgs['platform_display_name'] = df_vgs['platform'].map(platform_display_map)


In [276]:
# Data sample
df_vgs.head()


,name,platform,year_of_release,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,critic_score,critic_count,user_score,user_count,developer,rating,platform_display_name
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,Wii
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN,Nintendo Entertainment System
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E,Wii
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E,Wii
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN,Game Boy


### Populate Data Vault 2.0 Tables - Video Game Dataset:

In [277]:
import hashlib
from datetime import datetime
import psycopg2

# Define a function to generate HK from BK using MD5 hash, handling NaN (NULL)
def generate_hk(value: str) -> str:
    return hashlib.md5(value.encode('utf-8')).hexdigest()


In [278]:
# Check BK missing values (NaN, NULL)
print(f"'name' missing: {df_vgs['name'].isna().sum()}")
print(f"'platform' missing: {df_vgs['platform_display_name'].isna().sum()}")
print(f"'publisher' missing: {df_vgs['publisher'].isna().sum()}")
print(f"'developer' missing: {df_vgs['developer'].isna().sum()}")

'name' missing: 2
'platform' missing: 0
'publisher' missing: 54
'developer' missing: 6623


In [279]:
# First replace 'Unknown' string with None in both columns, that later i can fill
# all missing values with 'Unknown {row_index}'
df_vgs['publisher'] = df_vgs['publisher'].replace('Unknown', None)
df_vgs['developer'] = df_vgs['developer'].replace('Unknown', None)


In [280]:
# Fill missing values with 'Unknown {row_index}'
# 'name'
df_vgs['name'] = df_vgs.apply(
    lambda row: row['name'] if pd.notna(row['name']) else f'Unknown Game {row.name}',
    axis=1
)

# 'developer'
df_vgs['developer'] = df_vgs.apply(
    lambda row: row['developer'] if pd.notna(row['developer']) else f'Unknown Developer {row.name}',
    axis=1
)

# 'publisher'
df_vgs['publisher'] = df_vgs.apply(
    lambda row: row['publisher'] if pd.notna(row['publisher']) else f'Unknown Publisher {row.name}',
    axis=1
)


In [281]:
# Check which columns have multiple values separated by a comma,
# so we can later split them into separate rows for Data Vault hubs and links
def has_multiple_values(cell):
    return ',' in str(cell)

for col in ['developer', 'publisher', 'platform']:
    multi_count = df_vgs[col].apply(has_multiple_values).sum()
    print(f"{col} has {multi_count} rows with multiple values")


developer has 522 rows with multiple values
publisher has 14 rows with multiple values
platform has 0 rows with multiple values


In [282]:
# Columns to split
split_cols = ['developer', 'publisher']

# Function to split a column by comma and explode into separate rows
def split_column(df, column):
    df[column] = df[column].str.split(',')  # split by comma
    df = df.explode(column)                 # create a new row for each value
    df[column] = df[column].str.strip()     # remove extra spaces
    return df

# Apply the splitting for both developer and publisher
for col in split_cols:
    df_vgs = split_column(df_vgs, col)


In [283]:
# Check again if it worked
for col in split_cols:
    multi_count = df_vgs[col].apply(has_multiple_values).sum()
    print(f"{col} has {multi_count} rows with multiple values")


developer has 0 rows with multiple values
publisher has 0 rows with multiple values


In [284]:
# Generate Hub Keys for Game, Platform, Developer, Publisher
df_vgs['game_hk'] = df_vgs['name'].apply(generate_hk)
df_vgs['platform_hk'] = df_vgs['platform_display_name'].apply(generate_hk)
df_vgs['developer_hk'] = df_vgs['developer'].apply(generate_hk)
df_vgs['publisher_hk'] = df_vgs['publisher'].apply(generate_hk)


In [285]:
# Add load_date and record_source
load_date = datetime.now()
record_source = 'Video Games Sales'

for col in ['load_date', 'record_source']:
    df_vgs[col] = load_date if col == 'load_date' else record_source


In [286]:
# Prepare Hub tables and drop duplicates
hub_game = df_vgs[['game_hk', 'name', 'load_date', 'record_source']].drop_duplicates()
hub_game.columns = ['game_hk', 'game_name', 'load_date', 'record_source']

hub_platform = df_vgs[['platform_hk', 'platform_display_name', 'load_date', 'record_source']].drop_duplicates()
hub_platform.columns = ['platform_hk', 'platform_name', 'load_date', 'record_source']

hub_developer = df_vgs[['developer_hk', 'developer', 'load_date', 'record_source']].drop_duplicates()
hub_developer.columns = ['developer_hk', 'developer_name', 'load_date', 'record_source']

hub_publisher = df_vgs[['publisher_hk', 'publisher', 'load_date', 'record_source']].drop_duplicates()
hub_publisher.columns = ['publisher_hk', 'publisher_name', 'load_date', 'record_source']


In [287]:
# Prepare Satellite tables
sat_game_details = df_vgs[['game_hk', 'year_of_release', 'genre', 'rating', 'load_date', 'record_source']].copy()
sat_game_details.columns = ['game_hk', 'year_of_release', 'genre', 'rating', 'load_date', 'record_source']

sat_game_sales = df_vgs[['game_hk', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'global_sales', 'load_date', 'record_source']].copy()
sat_game_sales.columns = ['game_hk', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'global_sales', 'load_date', 'record_source']

sat_game_scores = df_vgs[['game_hk', 'critic_score', 'critic_count', 'user_score', 'user_count', 'load_date', 'record_source']].copy()
sat_game_scores.columns = ['game_hk', 'critic_score', 'critic_count', 'user_score', 'user_count', 'load_date', 'record_source']


In [288]:
# Define another function to generate HK (hash key) for Link tables by combining two BKs;
# Returns None (NULL in DB) if either key is missing
def generate_link_hk(key1, key2):
    return generate_hk(str(key1) + str(key2))  # concatenate as strings, then hash


In [289]:
# Prepare Link tables, drop duplicates
link_game_platform = df_vgs[['game_hk', 'platform_hk', 'load_date', 'record_source']].drop_duplicates()
link_game_platform['link_game_platform_hk'] = link_game_platform.apply(lambda x: generate_link_hk(x['game_hk'], x['platform_hk']), axis=1)
link_game_platform = link_game_platform[['link_game_platform_hk', 'game_hk', 'platform_hk', 'load_date', 'record_source']]

link_game_developer = df_vgs[['game_hk', 'developer_hk', 'load_date', 'record_source']].drop_duplicates()
link_game_developer['link_game_developer_hk'] = link_game_developer.apply(lambda x: generate_link_hk(x['game_hk'], x['developer_hk']), axis=1)
link_game_developer = link_game_developer[['link_game_developer_hk', 'game_hk', 'developer_hk', 'load_date', 'record_source']]

link_game_publisher = df_vgs[['game_hk', 'publisher_hk', 'load_date', 'record_source']].drop_duplicates()
link_game_publisher['link_game_publisher_hk'] = link_game_publisher.apply(lambda x: generate_link_hk(x['game_hk'], x['publisher_hk']), axis=1)
link_game_publisher = link_game_publisher[['link_game_publisher_hk', 'game_hk', 'publisher_hk', 'load_date', 'record_source']]


In [290]:
# Connect to PostgreSQL (I'm using my local DB as an example, no need to hide login info)
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="admin",
    password="admin"
)
cursor = conn.cursor()

In [291]:
# Define a function to insert a dataframe into a table
def insert_dataframe(df, table_name):
    df_clean = df.replace({np.nan: None})
    
    cols = ','.join(df_clean.columns)
    vals = ','.join(['%s'] * len(df_clean.columns))
    query = f"INSERT INTO {table_name} ({cols}) VALUES ({vals}) ON CONFLICT DO NOTHING;"
    
    for row in df_clean.values.tolist():
        cursor.execute(query, tuple(row))
    
    conn.commit()
    print(f"{len(df_clean)} rows inserted into {table_name}")

In [292]:
# Insert data into hubs
insert_dataframe(hub_game, 'hub_game')
insert_dataframe(hub_platform, 'hub_platform')
insert_dataframe(hub_developer, 'hub_developer')
insert_dataframe(hub_publisher, 'hub_publisher')

11564 rows inserted into hub_game
31 rows inserted into hub_platform
8159 rows inserted into hub_developer
836 rows inserted into hub_publisher


In [293]:
import numpy as np

# Convert year_of_release NaNs to None for database insertion (SMALLINT restriction in DB)
sat_game_details['year_of_release'] = sat_game_details['year_of_release'].replace({np.nan: None})


In [294]:
# Replace 'tbd' and other non-numeric values with None
for col in ['user_score', 'critic_score', 'critic_count', 'user_count']:
    if col in sat_game_scores.columns:
        sat_game_scores[col] = pd.to_numeric(sat_game_scores[col], errors='coerce')


In [295]:
# Insert data into satellites
insert_dataframe(sat_game_details, 'sat_game_details')
insert_dataframe(sat_game_sales, 'sat_game_sales')
insert_dataframe(sat_game_scores_clean, 'sat_game_scores')


17278 rows inserted into sat_game_details
17278 rows inserted into sat_game_sales
10087 rows inserted into sat_game_scores


In [296]:
# Insert data into links
insert_dataframe(link_game_platform, 'link_game_platform')
insert_dataframe(link_game_developer, 'link_game_developer')
insert_dataframe(link_game_publisher, 'link_game_publisher')


16715 rows inserted into link_game_platform
13917 rows inserted into link_game_developer
12015 rows inserted into link_game_publisher


In [297]:
# Close connection
cursor.close()
conn.close()
